### Setup Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

### Load the dataset

In [3]:
# Load the dataset
file_path = 'data/bank-full.csv'
bank_data = pd.read_csv(file_path, sep=';')

# Display the first few rows of the dataset to understand its structure
bank_data.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### Cleaning data

In [5]:
# Checking for missing values in the dataset
missing_data = bank_data.isnull().sum()

# Checking the distribution of the target variable "y"
target_distribution = bank_data['y'].value_counts()

# Displaying the results
missing_data, target_distribution


(age          0
 job          0
 marital      0
 education    0
 default      0
 balance      0
 housing      0
 loan         0
 contact      0
 day          0
 month        0
 duration     0
 campaign     0
 pdays        0
 previous     0
 poutcome     0
 y            0
 dtype: int64,
 y
 no     39922
 yes     5289
 Name: count, dtype: int64)

#### Encoding

In [7]:
# Encoding categorical variables using one-hot encoding for categorical features
bank_data_encoded = pd.get_dummies(bank_data, columns=[
    'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'
], drop_first=True)

# Encoding the target variable 'y' (yes=1, no=0)
bank_data_encoded['y'] = bank_data_encoded['y'].apply(lambda x: 1 if x == 'yes' else 0)

### Splitting, Scaling, Training, classification reports and Confusion matrices

In [ ]:

# Splitting the data into features (X) and target (y)
X = bank_data_encoded.drop(columns=['y'])
y = bank_data_encoded['y']

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling the features for better performance of distance-based classifiers (kNN, SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training and evaluating Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train_scaled, y_train)
y_pred_tree = decision_tree.predict(X_test_scaled)

# Training and evaluating Support Vector Machine (SVM)
svm = SVC(random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)

# Generating classification reports and confusion matrices for both models
tree_report = classification_report(y_test, y_pred_tree)
svm_report = classification_report(y_test, y_pred_svm)

# Confusion matrices
tree_conf_matrix = confusion_matrix(y_test, y_pred_tree)
svm_conf_matrix = confusion_matrix(y_test, y_pred_svm)

##tree_report, tree_conf_matrix, svm_report, svm_conf_matrix
# Printing classification reports in a more organized manner
print("Decision Tree Classification Report:\n")
print(tree_report)

print("Decision Tree Confusion Matrix:\n")
print(tree_conf_matrix)

print("SVM Classification Report:\n")
print(svm_report)

print("SVM Confusion Matrix:\n")
print(svm_conf_matrix)


### Metrics for comparison and visualization

In [ ]:

# Defining the models and their respective metrics for comparison
models = ['kNN', 'Logistic Regression', 'Decision Tree', 'SVM']
accuracy = [0.87, 0.86, 0.87, 0.90]
f1_score_class_1 = [0.46, 0.44, 0.45, 0.44]

# Creating a bar chart to compare accuracy and F1-score (for class 1)
x = np.arange(len(models))  # Label locations
width = 0.35  # Width of bars

# Plotting the accuracy and F1 scores
fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, accuracy, width, label='Accuracy')
rects2 = ax.bar(x + width/2, f1_score_class_1, width, label='F1-Score (Class 1)')

# Adding labels, title, and custom x-axis tick labels
ax.set_ylabel('Scores')
ax.set_title('Model Comparison: Accuracy and F1-Score (Class 1)')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Function to attach labels to bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}', xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

# Adding labels to the bars
autolabel(rects1)
autolabel(rects2)

# Displaying the plot
plt.tight_layout()
plt.show()


# Findings

### Business understanding of problem

El objetivo del análisis es predecir qué clientes son más propensos a suscribirse a un depósito a plazo, basándose en los datos de campañas de marketing anteriores. La capacidad de identificar estos clientes de manera precisa permite al banco enfocar sus esfuerzos de marketing en individuos que tienen más probabilidades de convertir, lo que resulta en una mayor eficiencia en el gasto de las campañas y un mejor compromiso con los clientes.

### Interpretation of descriptive and inferential statistics

Distribución de la Variable Objetivo: El conjunto de datos está altamente desbalanceado, con alrededor del 88% de los clientes que no se suscriben y solo el 12% que sí lo hacen. Este desequilibrio significa que los modelos pueden lograr una alta precisión al predecir principalmente la clase mayoritaria (no suscritos), pero pueden tener dificultades para detectar la clase minoritaria (suscritos).

Rendimiento de los Modelos:

k-Nearest Neighbors (kNN) y Regresión Logística alcanzaron aproximadamente un 87% de precisión, pero tuvieron dificultades para identificar correctamente a la clase minoritaria (clientes suscritos). Sus puntuaciones F1 para la clase minoritaria fueron de 0.46 y 0.44, respectivamente.

Árboles de Decisión también alcanzaron un 87% de precisión y tuvieron un rendimiento similar con una puntuación F1 de 0.45 para la clase minoritaria, mostrando una ligera mejora en el recall de la clase minoritaria en comparación con kNN y Regresión Logística.

Máquina de Vectores de Soporte (SVM) fue el modelo con mejor precisión general (90%), pero tuvo dificultades similares para identificar a los suscriptores, con una puntuación F1 de 0.44 para la clase minoritaria. Su alta precisión se debe principalmente a su buen rendimiento en la predicción de la clase mayoritaria (no suscritos).

### Findings

1- Alta Precisión en Todos los Modelos: Todos los modelos tuvieron un buen rendimiento en términos de precisión general. Sin embargo, dada la desproporción de clases, la precisión por sí sola no es una medida suficiente del rendimiento.

2- Dificultades con la Clase Minoritaria: Todos los modelos tuvieron dificultades para predecir a los clientes que se suscribieron al depósito a plazo. Las puntuaciones F1 para esta clase rondan entre 0.44 y 0.46, lo que indica que es difícil detectar verdaderos positivos (suscriptores). El bajo recall para la clase minoritaria sugiere que muchos suscriptores potenciales fueron pasados por alto.

3- Conclusiones Accionables:

    * El modelo de Máquina de Vectores de Soporte (SVM) es el mejor en términos de precisión general, por lo que es una buena opción si la prioridad del banco es el rendimiento global de la campaña en lugar de centrarse en un grupo reducido de posibles suscriptores.
    
    * Si el banco está particularmente interesado en mejorar la identificación de suscriptores potenciales (es decir, enfocándose en la clase minoritaria), deberían considerarse estrategias alternativas para mejorar el recall en ese grupo.

### Next steps and recommendations

Explorar Métodos de Ensamble: Métodos como Random Forests o Gradient Boosting Machines podrían explorarse para mejorar el rendimiento del modelo. Estos métodos suelen ser más robustos y pueden mejorar la precisión de la predicción para la clase minoritaria.

Estrategia de Negocio: El banco debería centrarse en optimizar las campañas para los clientes que son más propensos a suscribirse, utilizando las predicciones del modelo para dirigir los esfuerzos hacia los clientes con alta probabilidad de conversión. Al mismo tiempo, los ajustes en el proceso de entrenamiento de los modelos pueden ayudar a identificar mejor a los suscriptores potenciales que pudieron haber sido pasados por alto, asegurando que los esfuerzos de marketing sean inclusivos y efectivos.